# Titanic Survival

#### Grading:


- Code: 90 pts
- Markdown Documentation: 10 pts


We are going to study the survival rate of passengers on titanic and what variables affected survival.

Load the dataset in `titanic.xls`. It contains data on all the passengers that travelled on the Titanic.

## Imports.


In [15]:
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats as stats

## Pandas Display Options.

In [16]:
# Telling pandas not convert to html tags.
pd.set_option('display.html.table_schema', True)
# Max columns and rows to display.
pd.set_option('display.max_columns', 15)
pd.set_option('display.max_rows', 8)

In [17]:
from IPython.core.display import HTML
HTML(filename='../data/titanic.html')

Name,Labels,Units,Levels,Storage,NAs
pclass,,,3,integer,0
survived,Survived,,,double,0
name,Name,,,character,0
sex,,,2,integer,0
age,Age,Year,,double,263
sibsp,Number of Siblings/Spouses Aboard,,,double,0
parch,Number of Parents/Children Aboard,,,double,0
ticket,Ticket Number,,,character,0
fare,Passenger Fare,British Pound (\243),,double,1
cabin,,,187,integer,0


In [28]:
# you would need xlrd - pip install xlrd
t_file = pd.ExcelFile('../data/titanic.xls')
t_df = t_file.parse("titanic", header=None)
t_df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,pclass,survived,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,boat,body,home.dest
1,1,1,"Allen, Miss. Elisabeth Walton",female,29,0,0,24160,211.338,B5,S,2,NaN,"St Louis, MO"
2,1,1,"Allison, Master. Hudson Trevor",male,0.9167,1,2,113781,151.55,C22 C26,S,11,NaN,"Montreal, PQ / Chesterville, ON"
3,1,0,"Allison, Miss. Helen Loraine",female,2,1,2,113781,151.55,C22 C26,S,NaN,NaN,"Montreal, PQ / Chesterville, ON"
4,1,0,"Allison, Mr. Hudson Joshua Creighton",male,30,1,2,113781,151.55,C22 C26,S,NaN,135,"Montreal, PQ / Chesterville, ON"


### Women and children first?

*** 1. Use the `groupby` method to calculate the proportion of passengers that survived by sex. (10 pts)*** 

In [70]:
df_female = t_df.groupby(by = 3).get_group("female")
print(df_female[df_female.loc[:, 1] == 1].shape[0] / t_df.shape[0])

0.2587786259541985


In [57]:
df_male = t_df.groupby(by = 3).get_group("male")
total_male = df_male.shape[0]
total_m_survived = df_male[df_male.loc[:, 1] == 1].shape[0]
survival_rate_male = (total_m_survived / total_male)
print("Male passenger survival rate: {:.2%}".format(survival_rate_male))

Male passenger survival rate: 19.10%


In [60]:
df_female = t_df.groupby(by = 3).get_group("female")
total_female = df_female.shape[0]
total_f_survived = df_female[df_female.loc[:, 1] == 1].shape[0]
survival_rate_female = (total_f_survived / total_female)
print("Female passenger survival rate: {:.2%}".format(survival_rate_female))

Female passenger survival rate: 72.75%


*** 2. Calculate the same proportion, but by class and sex. (10 pts)*** 

In [67]:
df_male = t_df.groupby(by = 3).get_group("male")
df_male_1 = df_male.groupby(by = 0).get_group(1)
df_male_2 = df_male.groupby(by = 0).get_group(2)
df_male_3 = df_male.groupby(by = 0).get_group(3)
male_1_survived = df_male_1[df_male_1.loc[:, 1] == 1].shape[0]
male_2_survived = df_male_2[df_male_2.loc[:, 1] == 1].shape[0]
male_3_survived = df_male_3[df_male_3.loc[:, 1] == 1].shape[0]
total_male = df_male.shape[0]
print("Male and pclass 1 survival rate: {:.2%}".format(male_1_survived / total_male))
print("Male and pclass 2 survival rate: {:.2%}".format(male_2_survived / total_male))
print("Male and pclass 3 survival rate: {:.2%}".format(male_3_survived / total_male))

Male and pclass 1 survival rate: 7.24%
Male and pclass 2 survival rate: 2.97%
Male and pclass 3 survival rate: 8.90%


In [68]:
df_female = t_df.groupby(by = 3).get_group("female")
df_f_1 = df_female.groupby(by = 0).get_group(1)
df_f_2 = df_female.groupby(by = 0).get_group(2)
df_f_3 = df_female.groupby(by = 0).get_group(3)
f_1_survived = df_f_1[df_f_1.loc[:, 1] == 1].shape[0]
f_2_survived = df_f_2[df_f_2.loc[:, 1] == 1].shape[0]
f_3_survived = df_f_3[df_f_3.loc[:, 1] == 1].shape[0]
total_female = df_female.shape[0]
print("Female and pclass 1 survival rate: {:.2%}".format(f_1_survived / total_female))
print("Female and pclass 2 survival rate: {:.2%}".format(f_2_survived / total_female))
print("Female and pclass 3 survival rate: {:.2%}".format(f_3_survived / total_female))

Female and pclass 1 survival rate: 29.83%
Female and pclass 2 survival rate: 20.17%
Female and pclass 3 survival rate: 22.75%


*** 3. Create age categories: children (under 14 years), adolescents (14-20), adult (21-64), and senior(65+), and calculate survival proportions by age category, class and sex. (20 pts)***

In [72]:
display(t_df)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,pclass,survived,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,boat,body,home.dest
1,1,1,"Allen, Miss. Elisabeth Walton",female,29,0,0,24160,211.338,B5,S,2,NaN,"St Louis, MO"
2,1,1,"Allison, Master. Hudson Trevor",male,0.9167,1,2,113781,151.55,C22 C26,S,11,NaN,"Montreal, PQ / Chesterville, ON"
3,1,0,"Allison, Miss. Helen Loraine",female,2,1,2,113781,151.55,C22 C26,S,NaN,NaN,"Montreal, PQ / Chesterville, ON"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1306,3,0,"Zabour, Miss. Thamine",female,NaN,1,0,2665,14.4542,NaN,C,NaN,NaN,NaN
1307,3,0,"Zakarian, Mr. Mapriededer",male,26.5,0,0,2656,7.225,NaN,C,NaN,304,NaN
1308,3,0,"Zakarian, Mr. Ortin",male,27,0,0,2670,7.225,NaN,C,NaN,NaN,NaN
1309,3,0,"Zimmerman, Mr. Leo",male,29,0,0,315082,7.875,NaN,S,NaN,NaN,NaN


In [113]:
df_titanic = t_df.loc[1:, :]

In [199]:
def display_survival(_df_age_group, _age_type):
    for sex in ("male", "female"):
        df_sex = _df_age_group.groupby(by = 3).get_group(sex)
        total_people = df_sex.shape[0]
        print("*" * 15, sex[0].upper() + sex[1:], _age_type, "*" * 15)
        for pclass in (1, 2, 3):
            try:
                df_pclass = df_sex.groupby(by = 0).get_group(pclass)
                total_survived = df_pclass[df_pclass.loc[:, 1] == 1].shape[0]
                proportion_survived = total_survived / total_people
                print(sex[0].upper() + sex[1:], _age_type, "and pclass", pclass,
                      "survival rate: {:.2%}".format(proportion_survived))
            except:
                print(sex[0].upper() + sex[1:], _age_type, "and pclass", pclass,
                      "survival rate: data does not exist")
        print()

# Child Survival Proportions

In [200]:
display_survival(df_titanic[df_titanic.loc[:, 4] < 14], "child")

*************** Male child ***************
Male child and pclass 1 survival rate: 9.43%
Male child and pclass 2 survival rate: 20.75%
Male child and pclass 3 survival rate: 22.64%

*************** Female child ***************
Female child and pclass 1 survival rate: 0.00%
Female child and pclass 2 survival rate: 30.43%
Female child and pclass 3 survival rate: 32.61%



# Adolescent Survival Proportions

In [201]:
display_survival(pd.merge(df_titanic[df_titanic.loc[:, 4] >= 14],
                         df_titanic[df_titanic.loc[:, 4] < 21],
                         how = "inner"), "adolescent")

*************** Male adolescent ***************
Male adolescent and pclass 1 survival rate: 1.15%
Male adolescent and pclass 2 survival rate: 2.30%
Male adolescent and pclass 3 survival rate: 9.20%

*************** Female adolescent ***************
Female adolescent and pclass 1 survival rate: 23.81%
Female adolescent and pclass 2 survival rate: 19.05%
Female adolescent and pclass 3 survival rate: 30.16%



# Adult Survival Proportions

In [202]:
display_survival(pd.merge(df_titanic[df_titanic.loc[:, 4] >= 21],
                   df_titanic[df_titanic.loc[:, 4] < 65],
                   how = "inner"), "adult")

*************** Male adult ***************
Male adult and pclass 1 survival rate: 9.09%
Male adult and pclass 2 survival rate: 1.98%
Male adult and pclass 3 survival rate: 7.71%

*************** Female adult ***************
Female adult and pclass 1 survival rate: 40.29%
Female adult and pclass 2 survival rate: 23.74%
Female adult and pclass 3 survival rate: 13.67%



# Senior Survival Proportions

In [203]:
display_survival(df_titanic[df_titanic.loc[:, 4] >= 65], "senior")

*************** Male senior ***************
Male senior and pclass 1 survival rate: 8.33%
Male senior and pclass 2 survival rate: 0.00%
Male senior and pclass 3 survival rate: 0.00%

*************** Female senior ***************
Female senior and pclass 1 survival rate: 100.00%
Female senior and pclass 2 survival rate: data does not exist
Female senior and pclass 3 survival rate: data does not exist

